In [59]:
import pandas as pd
import pypsa
import matplotlib.pyplot as plt

ArgumentError: ArgumentError: Package pandas not found in current path.
- Run `import Pkg; Pkg.add("pandas")` to install the pandas package.

### Creating a one bus for the Netherlands

In [60]:
network = pypsa.Network()
hours_in_2015 = pd.date_range('2015-01-01 00:00Z',
                              '2015-12-31 23:00Z',
                              freq='h')

network.set_snapshots(hours_in_2015.values)


network.add("Bus", "electricity bus", carrier="electricity")
network.add("Carrier", "electricity")  

network.snapshots

UndefVarError: UndefVarError: `pypsa` not defined

#### Getting the demand for the Netherlands and adding it to the bus

In [61]:
# Define file path
file_path = "Data/electricity_demand.csv"

# Load dataset
data = pd.read_csv(file_path, sep=';', parse_dates=['utc_time'], index_col='utc_time')
data.index = pd.to_datetime(data.index, utc=True)
# Convert index to datetime format
data.index = pd.to_datetime(data.index, utc=True)

start_date = '2015-01-01 00:00:00+00:00'
end_date = '2015-12-31 23:00:00+00:00'

data = data[start_date:end_date]
country = 'NLD'
data = data[country]
print(data.head())

ErrorException: syntax: character literal contains multiple characters

In [62]:
# add load to the bus
network.add("Load",
            "load",
            bus="electricity bus",
            p_set = data.values)

UndefVarError: UndefVarError: `data` not defined

In [63]:
network.loads_t.p_set

UndefVarError: UndefVarError: `network` not defined

#### Adding different generation types 

In [64]:
def annuity(n,r):
    """ Calculate the annuity factor for an asset with lifetime n years and
    discount rate  r """

    if r > 0:
        return r/(1. - 1./(1.+r)**n)
    else:
        return 1/n
    
print(annuity(25,0.07))
print(annuity(25,0.07)*425000*(1+0.03))

ErrorException: syntax: extra token "annuity" after end of expression

In [65]:
# add the different carriers, only gas emits CO2
network.add("Carrier", "coal", co2_emissions=0.363) # in /MWh_tt_CO2h
network.add("Carrier", "offshorewind")
network.add("Carrier", "onshorewind")
network.add("Carrier", "solar")
network.add("Carrier", "nuclear")
network.add("Carrier", "gas", co2_emissions=0.22) # in /MWh_tt_CO2h



# add onshore wind generator
df_onshorewind = pd.read_csv('Data/onshore_wind_1979-2017.csv', sep=';', index_col=0)
df_onshorewind.index = pd.to_datetime(df_onshorewind.index)
CF_wind_on = df_onshorewind[country][[hour.strftime("%Y-%m-%dT%H:%M:%SZ") for hour in network.snapshots]]
capital_cost_onshorewind = annuity(30,0.07)*1088000*(1+0.033) # in €/MW
network.add("Generator",
            "onshorewind",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="onshorewind",
            #p_nom_max=7000, # maximum capacity can be limited due to environmental constraints
            capital_cost = capital_cost_onshorewind,
            marginal_cost = 0,
            p_max_pu = CF_wind_on.values)

# add offshore wind generator
df_offshorewind = pd.read_csv('Data/offshore_wind_1979-2017.csv', sep=';', index_col=0)
df_offshorewind.index = pd.to_datetime(df_offshorewind.index)
CF_wind_off = df_offshorewind[country][[hour.strftime("%Y-%m-%dT%H:%M:%SZ") for hour in network.snapshots]]
capital_cost_offshorewind = annuity(30,0.07)*2084520*(1+0.033) # in €/MW
network.add("Generator",
            "offshorewind",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="offshorewind",
            #p_nom_max=7000, # maximum capacity can be limited due to environmental constraints
            capital_cost = capital_cost_offshorewind,
            marginal_cost = 0,
            p_max_pu = CF_wind_off.values)

# add solar PV generator
df_solar = pd.read_csv('Data/pv_optimal.csv', sep=';', index_col=0)
df_solar.index = pd.to_datetime(df_solar.index)
CF_solar = df_solar[country][[hour.strftime("%Y-%m-%dT%H:%M:%SZ") for hour in network.snapshots]]
capital_cost_solar = annuity(25,0.07)*600290*(1+0.03) # in €/MW
network.add("Generator",
            "solar",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="solar",
            #p_nom_max=1000, # maximum capacity can be limited due to environmental constraints
            capital_cost = capital_cost_solar,
            marginal_cost = 0,
            p_max_pu = CF_solar.values)

# add Gas CC W/o ccts
capital_cost_CCGT = annuity(30,0.07)*1300000*(1+0.033) # in €/MW
fuel_cost = 21.6 # in €/MWh_th
efficiency = 0.58 # MWh_elec/MWh_th
VOM_gas = 4 # in €/MWh
marginal_cost_CCGT = VOM_gas + fuel_cost/efficiency # in €/MWh_el
network.add("Generator",
            "CCGT",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="gas",
            #p_nom_max=7000,
            capital_cost = capital_cost_CCGT,
            marginal_cost = marginal_cost_CCGT)


# add coal power plant  generator coal IGCC W/o ccts
capital_cost_coal = annuity(40,0.07)*3088000*(1+0.03) # in €/MW
fuel_cost = 8.4 # in €/MWh_th
efficiency = 0.392 # MWh_elec/MWh_th
VOM_coal = 6 # in €/MWh
marginal_cost_coal = VOM_coal + fuel_cost/efficiency # in €/MWh_el
network.add("Generator",
            "coal",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="coal",
            #p_nom_max=7000,
            capital_cost = capital_cost_coal,
            marginal_cost = marginal_cost_coal)


# add nuclear power plant generator
capital_cost_nuclear = annuity(45,0.07)*8594000*(1+0.027) # in €/MW
fuel_cost = 3 # in €/MWh_th 
efficiency = 0.33 # MWh_elec/MWh_th
VOM_nuclear = 10 # in €/MWh
marginal_cost_nuclear = VOM_nuclear + fuel_cost/efficiency # in €/MWh_el
network.add("Generator",
            "nuclear",
            bus="electricity bus",
            p_nom_extendable=True,
            carrier="nuclear",
            #p_nom_max=7000,
            capital_cost = capital_cost_nuclear,
            marginal_cost = marginal_cost_nuclear)

UndefVarError: UndefVarError: `network` not defined

In [66]:
network.generators_t.p_max_pu

UndefVarError: UndefVarError: `network` not defined

In [67]:
network.optimize(solver_name='gurobi')

ErrorException: syntax: character literal contains multiple characters

In [68]:
print(network.objective/1000000) #in 10^6 €

UndefVarError: UndefVarError: `network` not defined

In [69]:
print(network.objective/network.loads_t.p.sum()) # EUR/MWh

UndefVarError: UndefVarError: `network` not defined

In [70]:
network.generators.p_nom_opt # in MW

UndefVarError: UndefVarError: `network` not defined

In [71]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)  # 1 row, 2 columns

# --- Winter Week (January) ---
axes[0].plot(network.loads_t.p['load'][0:168], color='black', label='demand')
axes[0].plot(network.generators_t.p['onshorewind'][0:168], color='blue', label='onshore wind')
axes[0].plot(network.generators_t.p['offshorewind'][0:168], color='green', label='offshore wind')
axes[0].plot(network.generators_t.p['solar'][0:168], color='orange', label='solar')
axes[0].plot(network.generators_t.p['CCGT'][0:168], color='brown', label='gas (CCGT)')
axes[0].plot(network.generators_t.p['coal'][0:168], color='red', label='coal')
axes[0].plot(network.generators_t.p['nuclear'][0:168], color='pink', label='nuclear')
axes[0].legend(fancybox=True, shadow=True, loc='best')
axes[0].set_title('Electricity production in one week in January')
axes[0].tick_params(axis='x', rotation=45)

# --- Summer Week (July) ---
axes[1].plot(network.loads_t.p['load'][4344:4512], color='black', label='demand')
axes[1].plot(network.generators_t.p['onshorewind'][4344:4512], color='blue', label='onshore wind')
axes[1].plot(network.generators_t.p['offshorewind'][4344:4512], color='green', label='offshore wind')
axes[1].plot(network.generators_t.p['solar'][4344:4512], color='orange', label='solar')
axes[1].plot(network.generators_t.p['CCGT'][4344:4512], color='brown', label='gas (CCGT)')
axes[1].plot(network.generators_t.p['coal'][4344:4512], color='red', label='coal')
axes[1].plot(network.generators_t.p['nuclear'][4344:4512], color='pink', label='nuclear')
axes[1].legend(fancybox=True, shadow=True, loc='best')
axes[1].set_title('Electricity production in one week in July')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

UndefVarError: UndefVarError: `True` not defined

## Electricity mix

In [72]:
labels = ['onshore wind',
          'solar',
          'gas (CCGT)',
          'Offshore wind',
          'Coal',
          'Nuclear']
sizes = [network.generators_t.p['onshorewind'].sum(),
         network.generators_t.p['solar'].sum(),
         network.generators_t.p['CCGT'].sum(),
         network.generators_t.p['offshorewind'].sum(),
        network.generators_t.p['coal'].sum(),
        network.generators_t.p['nuclear'].sum()]

colors=['blue', 'orange', 'brown', 'green', 'red', 'pink']

plt.pie(sizes,
        colors=colors,
        labels=labels,
        wedgeprops={'linewidth':0})
plt.axis('equal')

plt.title('Electricity mix', y=1.07)

ErrorException: syntax: character literal contains multiple characters

In [73]:
# Group dispatch by carrier manually
dispatch = network.generators_t.p
carrier_map = network.generators.carrier

# Match generator names to carriers
dispatch.columns = carrier_map[dispatch.columns].values

UndefVarError: UndefVarError: `network` not defined

In [74]:
# Sum all generators by carrier
carrier_dispatch = dispatch.groupby(axis=1, level=0).sum()
# Duration curves
carrier_dispatch.apply(lambda x: x.sort_values(ascending=False).reset_index(drop=True)).plot(
    figsize=(12, 6), title="Generation Duration Curves")
plt.ylabel("Power Output [MW]")
plt.xlabel("Sorted Hours")
plt.grid(True)
plt.tight_layout()
plt.show()

UndefVarError: UndefVarError: `dispatch` not defined

## CO2 limit 

In [75]:
co2_limit=13000000#tonCO2
network.add("GlobalConstraint",
            "co2_limit_13MT",
            type="primary_energy",
            carrier_attribute="co2_emissions",
            sense="<=",
            constant=co2_limit)

#network.optimize(solver_name='gurobi')
network.optimize(solver_name='gurobi', solver_options={'OutputFlag': 0})



labels = ['onshore wind',
          'solar',
          'gas (CCGT)',
          'Offshore wind',
          'Coal',
          'Nuclear']
sizes = [network.generators_t.p['onshorewind'].sum(),
         network.generators_t.p['solar'].sum(),
         network.generators_t.p['CCGT'].sum(),
         network.generators_t.p['offshorewind'].sum(),
        network.generators_t.p['coal'].sum(),
        network.generators_t.p['nuclear'].sum()]

colors=['blue', 'orange', 'brown', 'green', 'red', 'pink']    

plt.pie(sizes,
        colors=colors,
        labels=labels,
        wedgeprops={'linewidth':0})
plt.axis('equal')

plt.title('Electricity mix', y=1.07)

network.generators.p_nom_opt # in MW

UndefVarError: UndefVarError: `network` not defined

In [76]:
co2_limits = np.linspace(0, 19.9e6, 10)  


energy_mixes = []
for co2_limit in co2_limits:
    
    network.remove("GlobalConstraint", "co2_limit") 
    network.add("GlobalConstraint",
                "co2_limit",  
                type="primary_energy",
                carrier_attribute="co2_emissions",
                sense="<=",
                constant=co2_limit)
    network.optimize(solver_name='gurobi', solver_options={'OutputFlag': 0})
    sizes = [network.generators_t.p['onshorewind'].sum(),
             network.generators_t.p['solar'].sum(),
             network.generators_t.p['CCGT'].sum(),
             network.generators_t.p['offshorewind'].sum(),
             network.generators_t.p['coal'].sum(),
             network.generators_t.p['nuclear'].sum()]
    energy_mixes.append(sizes)

energy_mixes = np.array(energy_mixes)


# Plot
labels = ['onshore wind', 'solar', 'gas (CCGT)', 'Offshore wind', 'Coal', 'Nuclear']
colors = ['blue', 'orange', 'brown', 'green', 'red', 'pink']
fig, ax1 = plt.subplots(figsize=(12, 7))
x = np.arange(len(co2_limits)) 
bar_width = 0.15 
for i, label in enumerate(labels):
    ax1.bar(x + i * bar_width, energy_mixes[:, i], bar_width, label=label, color=colors[i])
ax1.set_xlabel('CO2 Limit (tonCO2)')
ax1.set_ylabel('Energy Production (MWh)')
ax1.set_title('Energy Mix vs CO2 Limit')
ax1.set_xticks(x + bar_width * 2) 
ax1.set_xticklabels([f'{co2_limit / 1e6:.1f} Mt' for co2_limit in co2_limits]) 
ax1.legend(title="Energy Sources")
fig.tight_layout()

plt.show()


UndefVarError: UndefVarError: `np` not defined

In [77]:
print(network.objective/1000000) #in 10^6 €

UndefVarError: UndefVarError: `network` not defined

In [78]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)  # 1 row, 2 columns

# --- Winter Week (January) ---
axes[0].plot(network.loads_t.p['load'][0:168], color='black', label='demand')
axes[0].plot(network.generators_t.p['onshorewind'][0:168], color='blue', label='onshore wind')
axes[0].plot(network.generators_t.p['offshorewind'][0:168], color='green', label='offshore wind')
axes[0].plot(network.generators_t.p['solar'][0:168], color='orange', label='solar')
axes[0].plot(network.generators_t.p['CCGT'][0:168], color='brown', label='gas (CCGT)')
axes[0].plot(network.generators_t.p['coal'][0:168], color='red', label='coal')
axes[0].plot(network.generators_t.p['nuclear'][0:168], color='pink', label='nuclear')
axes[0].legend(fancybox=True, shadow=True, loc='best')
axes[0].set_title('Electricity production in one week in January')
axes[0].tick_params(axis='x', rotation=45)

# --- Summer Week (July) ---
axes[1].plot(network.loads_t.p['load'][4344:4512], color='black', label='demand')
axes[1].plot(network.generators_t.p['onshorewind'][4344:4512], color='blue', label='onshore wind')
axes[1].plot(network.generators_t.p['offshorewind'][4344:4512], color='green', label='offshore wind')
axes[1].plot(network.generators_t.p['solar'][4344:4512], color='orange', label='solar')
axes[1].plot(network.generators_t.p['CCGT'][4344:4512], color='brown', label='gas (CCGT)')
axes[1].plot(network.generators_t.p['coal'][4344:4512], color='red', label='coal')
axes[1].plot(network.generators_t.p['nuclear'][4344:4512], color='pink', label='nuclear')
axes[1].legend(fancybox=True, shadow=True, loc='best')
axes[1].set_title('Electricity production in one week in July')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

UndefVarError: UndefVarError: `True` not defined

Part C. Plotting Average Capacity and variability for every generator

In [79]:
#calculating the average capacity for each generator using different years
Country = 'NLD'

data_pv = pd.read_csv('Data/pv_optimal.csv', sep=';', index_col=0)
data_pv.index = pd.to_datetime(data_pv.index)
CF_pv = data_pv[country]['1979-1-01T00:00:00Z':'2017-12-31T23:00:00Z']

data_onshore_wind = pd.read_csv('Data/onshore_wind_1979-2017.csv', sep=';', index_col=0)
data_onshore_wind.index = pd.to_datetime(data_onshore_wind.index)
CF_onshore_wind = data_onshore_wind[country]['1979-1-01T00:00:00Z':'2017-12-31T23:00:00Z']

data_offshore_wind = pd.read_csv('Data/offshore_wind_1979-2017.csv', sep=';', index_col=0)
data_offshore_wind.index = pd.to_datetime(data_offshore_wind.index)
CF_offshore_wind = data_offshore_wind[country]['1979-1-01T00:00:00Z':'2017-12-31T23:00:00Z']

CF_all = pd.concat([CF_pv.rename('PV'),
                    CF_onshore_wind.rename('onshore wind'),
                    CF_offshore_wind.rename('offshore wind')], axis=1)

average = CF_all.mean()
std_dev = CF_all.std()

plt.figure(figsize=(10, 6))
plt.bar(average.index, average.values, yerr=std_dev.values, capsize=5, alpha = 0.7)
plt.ylabel('Capacity Factor')
plt.title('Generator Capacity Sensitivity to Interannual Variability')
plt.grid(True, linestyle = '--', alpha = 0.5)
plt.tight_layout()
plt.show()






ErrorException: syntax: character literal contains multiple characters

Part D. Adding Storage Technology

In [80]:
network.add("Carrier", "battery")
network.add("Carrier", "Pumped Hydro")

# add storage units (battery and pumped hydro)
network.add("StorageUnit", "battery", 
            bus="electricity bus",
            p_nom=300000,
            max_hours=4.5,
            p_nom_extendable = True,
            efficiency_store=0.9,
            efficiency_dispatch=0.9,
            capital_cost=350000,
            marginal_cost=0,
            carrier="battery",
            overwrite = True)

network.add("StorageUnit", "Pumped Hydro", 
            bus="electricity bus",
            p_nom=500,
            p_nom_extendable = True,
            max_hours=12,
            efficiency_store=0.9,
            efficiency_dispatch=0.9,
            capital_cost=2000000,
            marginal_cost=0,
            carrier="Pumped Hydro",
            overwrite = True)


UndefVarError: UndefVarError: `network` not defined

In [81]:
network.storage_units.head(3)

UndefVarError: UndefVarError: `network` not defined

In [82]:
network.optimize(solver_name='gurobi')

ErrorException: syntax: character literal contains multiple characters

In [83]:
p_nom_opt_batt = network.storage_units.loc["battery", "p_nom_opt"]
p_nom_opt_pump = network.storage_units.loc["Pumped Hydro", "p_nom_opt"]
print(p_nom_opt_batt)
print(p_nom_opt_pump)

UndefVarError: UndefVarError: `network` not defined

In [84]:
network.storage_units_t.state_of_charge["battery"].loc["2015-06-01":"2015-06-07"].plot()
plt.title("Battery State of Charge (Week of June 2017)")
plt.ylabel("Energy [MWh]")
plt.grid()
plt.show()


UndefVarError: UndefVarError: `network` not defined

Part E: target for decarbonization

In [85]:
co2_limit = 13e6  

co2_prices = []


network.remove("GlobalConstraint", "co2_limit")


network.add("GlobalConstraint",
            "co2_limit",
            type="primary_energy",
            carrier_attribute="co2_emissions",
            sense="<=",
            constant=co2_limit)

network.optimize(solver_name='gurobi', solver_options={'OutputFlag': 0})

co2_price = network.global_constraints.mu
co2_prices.append(co2_price)


print(f"CO2 Limit: {co2_limit / 1e6:.1f} Mt, CO2 Price: {co2_price}")


UndefVarError: UndefVarError: `network` not defined

In [86]:
co2_limits = np.linspace(0, 19.9e6, 10)

co2_prices = []

for co2_limit in co2_limits:
    network.remove("GlobalConstraint", "co2_limit")
    
    network.add("GlobalConstraint",
                "co2_limit",
                type="primary_energy",
                carrier_attribute="co2_emissions",
                sense="<=",
                constant=co2_limit)
    
    network.optimize(solver_name='gurobi', solver_options={'OutputFlag': 0})
    
    co2_price = network.global_constraints.mu
    co2_prices.append(co2_price)

print("CO2 Prices for each CO2 limit:")
for co2_limit, co2_price in zip(co2_limits, co2_prices):
    print(f"CO2 Limit: {co2_limit / 1e6:.1f} Mt, CO2 Price: {co2_price} EUR/ton")


fig, ax1 = plt.subplots(figsize=(10, 6))
ax1.plot(co2_limits / 1e6, co2_prices, marker='o', linestyle='-', color='b')
ax1.set_xlabel('CO2 Limit (Mton CO2)', fontsize=12)
ax1.set_ylabel('CO2 Price (EUR/ton CO2)', fontsize=12)
ax1.set_title('CO2 Price vs CO2 Limit', fontsize=14)
ax1.grid(True)

plt.tight_layout()
plt.show()


UndefVarError: UndefVarError: `np` not defined

In [87]:
import numpy as np

co2_limits = np.linspace(0, 19.9e6, 10)
co2_prices = []

for co2_limit in co2_limits:
    network.remove("GlobalConstraint", "co2_limit")
    network.add("GlobalConstraint",
                "co2_limit",
                type="primary_energy",
                carrier_attribute="co2_emissions",
                sense="<=",
                constant=co2_limit)
    network.optimize(solver_name='gurobi', solver_options={'OutputFlag': 0})
    try:
        mu = network.global_constraints.at["co2_limit_13MT", "mu"]
        price = float(mu) if mu is not None else np.nan
    except:
        price = np.nan
    co2_prices.append(price)

co2_limits = np.array(co2_limits)
co2_prices = np.array(co2_prices)

valid = ~np.isnan(co2_prices)
co2_limits_clean = co2_limits[valid]
co2_prices_clean = co2_prices[valid]

sorted_indices = np.argsort(co2_prices_clean)
co2_prices_sorted = co2_prices_clean[sorted_indices]
co2_limits_sorted = co2_limits_clean[sorted_indices]

if len(co2_prices_sorted) >= 2:
    target_price = -88
    co2_limit_target = np.interp(target_price, co2_prices_sorted, co2_limits_sorted)
    print(f"{co2_limit_target / 1e6:.1f} Mt")
else:
    print("Not enough valid values")


ArgumentError: ArgumentError: Package numpy not found in current path.
- Run `import Pkg; Pkg.add("numpy")` to install the numpy package.